# FIFA World Cup Match Data — Cleaning Notebook

## About This Notebook

`WorldCupMatches.csv` is the most complex of the three datasets — 4572 rows covering every match played across 20 World Cups, with columns for teams, scores, venues, officials, and win conditions. A lot can go wrong in 84 years of historical data.

**What this notebook does:**

- Drops 128 fully null rows and removes 16 duplicate match entries (keyed on `MatchID`)
- Fills the one remaining null attendance value — Germany vs Algeria, 2014 — by looking it up on Transfermarkt (43063)
- Converts numerical columns from float64 to `uint8`/`uint16`/`uint32` and parses the Datetime column from strings to `datetime64`
- Fixes inconsistent month names in the datetime strings before parsing
- Standardises 9 Stage values (e.g. unifying group stage and third-place naming across eras)
- Maps deprecated country names and updates the corresponding team initials and Win Conditions entries
- Rebuilds the Win Conditions column with a consistent format: Draw, `[Team] win`, `[Team] win after extra time`, or `[Team] win on penalties (X - Y)`
- Cross-validates Year vs parsed Datetime (0 mismatches found)

**One thing to flag:**  
The Win Conditions logic reports 4 apparent mismatches. Manual inspection shows these are false positives caused by how the regex handles country names with parenthetical notation (e.g. `Germany (Germany FR)`). The data is correct; the detection logic is the issue, and it's documented as such.

**Tools:** pandas · numpy · re


Importing needed libraries numpy, pandas, regex

In [ ]:
import numpy as np
import pandas as pd
import re

WorldCupMatches csv cleaning

Reading WorldCupMatches csv with utf-8 encoding

In [ ]:
worldcup_matches = pd.read_csv('../data/WorldCupMatches.csv', encoding='utf-8')

Exploring WorldCupMatches table

A glance on the first 10 enteries of the table

In [ ]:
worldcup_matches.head(10)

Table info

In [ ]:
worldcup_matches.info()

Many null enteries in all columns

Stats for columns with numerical values

In [ ]:
worldcup_matches.describe()

Numerical mean, min, and max look reasonable.

In [ ]:
worldcup_matches.describe(include='object')

Values frequencies look ok

Updating Win condition column name to preseve consistency with other columns names cases

In [ ]:
worldcup_matches = worldcup_matches.rename(columns={'Win conditions': 'Win Conditions'})

Checking number of null cells in table 

In [ ]:
worldcup_matches.isnull().sum()

Found many null cells 

Checking enteries where Year column is null

In [ ]:
worldcup_matches.loc[worldcup_matches.Year.isnull()]

looks like enteries where column Year is null all other columns are null

Checking this assupmtion:

In [ ]:
if worldcup_matches.loc[worldcup_matches.Year.isnull()].isnull().all().all():
    print("Where Year is Null, all cells are Null")
else:
    print("Where Year is Null, there are non-Null cells")

All cells are null in this case, then removing those null enteries

In [ ]:
worldcup_matches = worldcup_matches.dropna(subset=['Year'])

Checking again number of null cells in table after null enteries are dropped

In [ ]:
worldcup_matches.isnull().sum()

Found 2 null cells exist in the table, they are in Attendance column

Checking enteries containg those null cells:

In [ ]:
worldcup_matches.loc[worldcup_matches.Attendance.isnull()]

2 null cells belong to duplicated enteries

Hence, checking first the duplicated enteries and fix this issue first, then return back to the null cell matter

In [ ]:
worldcup_matches.duplicated().sum()

Found 16 duplicated enteries.

Checking number of duplicated MatchIDs

In [ ]:
worldcup_matches.duplicated(subset=['MatchID']).sum()

All duplicated enteries are MatchID duplicated 

In [ ]:
worldcup_matches.loc[worldcup_matches.MatchID.duplicated(keep=False)]

Dropping duplicated enteries by MatchId

In [ ]:
worldcup_matches = worldcup_matches.drop_duplicates(subset=['MatchID'])

Checking number of duplicated enteries afte rdropping MatchID duplicates

In [ ]:
worldcup_matches.duplicated().sum()

All duplicates are removed

Returning to null check, and checking null cells count

In [ ]:
worldcup_matches.isnull().sum()

Now null cells count is 1. Was 2 before but they were basically same cell as the other one was duplicated

In [ ]:
worldcup_matches.loc[worldcup_matches.Attendance.isnull()]

It's a match, that has no attendance recorded in the table

Checking Germany - Algeria match in word cup 2014 page on transfermarkt.com, getting the attendance count for htis match, and record it in this table
refernce: https://www.transfermarkt.com/germany_algeria/index/spielbericht/2462534

In [ ]:
worldcup_matches.loc[worldcup_matches.Attendance.isnull(), 'Attendance'] = 43063

Checking null cells count after fixing Attendance null cell

In [ ]:
worldcup_matches.isnull().sum()

Found no null cells. Null issues fixed

Updating data types for numerical columns, as they are all integr values, and doesn't need to be float. Also optimizing bits  in the data types depending on the needed number of bits, to optimize memory utilisation

In [ ]:
worldcup_matches['Year'] = worldcup_matches['Year'].astype(np.uint16)
worldcup_matches['Home Team Goals'] = worldcup_matches['Home Team Goals'].astype(np.uint8)
worldcup_matches['Away Team Goals'] = worldcup_matches['Away Team Goals'].astype(np.uint8)
worldcup_matches['Attendance'] = worldcup_matches['Attendance'].astype(np.uint32)
worldcup_matches['Half-time Home Goals'] = worldcup_matches['Half-time Home Goals'].astype(np.uint8)
worldcup_matches['Half-time Away Goals'] = worldcup_matches['Half-time Away Goals'].astype(np.uint8)
worldcup_matches['RoundID'] = worldcup_matches['RoundID'].astype(np.uint32)
worldcup_matches['MatchID'] = worldcup_matches['MatchID'].astype(np.uint32)

Cheking dtypes of columns after modification

In [ ]:
worldcup_matches.dtypes

Datetime column has object dtype insead of datetime dtype. Hence needed to be updated

Checking the unique values of Datetime columns, to fix the irregularities

In [ ]:
worldcup_matches.Datetime.unique()

Found some irregularities in months names.

Updating months name to be consistent and comapitble with datetime dtype format

In [ ]:
months_abb = {
    'January': 'Jan',
    'February': 'Feb',
    'March': 'Mar',
    'April': 'Apr',
    'May': 'May',
    'June': 'Jun',
    'July': 'Jul',
    'August': 'Aug',
    'September': 'Sep',
    'October': 'Oct',
    'November': 'Nov',
    'December': 'Dec',
}
months_pattern = '|'.join(months_abb.keys())
worldcup_matches.Datetime = worldcup_matches.Datetime.str.replace(months_pattern, lambda x: months_abb[x.group()], regex=True).str.strip()
worldcup_matches.Datetime.unique()

Months names updated well

Datetime column dtype update

In [ ]:
worldcup_matches['Datetime'] = pd.to_datetime(worldcup_matches['Datetime'], format='%d %b %Y - %H:%M')
worldcup_matches.dtypes

Basic checks are done.

Checking column by column values, to spot any irrregularities or logical issues in the table

Checking if Year column values are same as values of year in Datetime column

In [ ]:
(worldcup_matches.Datetime.dt.year != worldcup_matches.Year).sum()

No mismatches between both.

Checking half time goals not to be bigger than total goals

In [ ]:
invalid_halftime = (
    (worldcup_matches['Half-time Home Goals'] > worldcup_matches['Home Team Goals']) |
    (worldcup_matches['Half-time Away Goals'] > worldcup_matches['Away Team Goals'])
)
invalid_halftime.sum()

All half time goals looks like ok

Checking unique values in Stage column

In [ ]:
worldcup_matches.Stage.unique()

Found irregular defeinition of groups, round of 16, and third place match. As they were named differently in different editions of the world cup

In [ ]:
rounds = {
    'Group 1': 'Group A',
    'Group 2': 'Group B',
    'Group 3': 'Group C',
    'Group 4': 'Group D',
    'Group 5': 'Group E',
    'Group 6': 'Group F',
    'Match for third place': 'Third place',
    'Play-off for third place': 'Third place',
    'Preliminary round': 'Round of 16',
    'First round': 'Round of 16',
}

worldcup_matches = worldcup_matches.replace(rounds)

Checking Stage values after fix

In [ ]:
worldcup_matches.Stage.unique()

Stage column values fixed

Checking Stadium values

In [ ]:
worldcup_matches.Stadium.unique()

Checking city unique values

In [ ]:
worldcup_matches.City.unique()

Checking unique values of all teams mentioned in home or away team names

In [ ]:
pd.concat([worldcup_matches['Home Team Name'], worldcup_matches['Away Team Name']]).unique()

Updating countries names, and modify old countries names in Home Team Name, Away Team Name, Win Condition columns, to make them consistent with the predecessor countries names

In [ ]:
countries = {
    'Germany FR': 'Germany (Germany FR)',
    'German DR': 'Germany (German DR)',
    'Soviet Union': 'Russia (Soviet Union)',
    'Czechoslovakia': 'Czech Republic (Czechoslovakia)',
    'rn">United Arab Emirates': 'United Arab Emirates',
    'rn">Republic of Ireland': 'Republic of Ireland',
    'rn">Trinidad and Tobago': 'Trinidad and Tobago',
    'rn">Serbia and Montenegro': 'Serbia',
    'Yugoslavia': 'Serbia (Yugoslavia)',
    'rn">Bosnia and Herzegovina': 'Bosnia and Herzegovina',
    'Dutch East Indies': 'Indonesia (Dutch East Indies)',
    'Zaire': 'Congo DR (Zaire)',
    'Iran': 'IR Iran',
}

countries_pattern = '|'.join(countries.keys())
worldcup_matches['Home Team Name'] = worldcup_matches['Home Team Name'].str.replace(countries_pattern, lambda x: countries[x.group()], regex=True).str.strip()
worldcup_matches['Away Team Name'] = worldcup_matches['Away Team Name'].str.replace(countries_pattern, lambda x: countries[x.group()], regex=True).str.strip()
worldcup_matches['Win Conditions'] = worldcup_matches['Win Conditions'].str.replace(countries_pattern, lambda x: countries[x.group()], regex=True).str.strip()

Checking teams names values after fixes

In [ ]:
teams_names = pd.concat([worldcup_matches['Home Team Name'], worldcup_matches['Away Team Name']]).unique()
teams_names

Checking max values in Home Team Goals and Away Team Goals to make sure they exist and logical

In [ ]:
worldcup_matches[worldcup_matches['Home Team Goals']==worldcup_matches['Home Team Goals'].max()]

In [ ]:
worldcup_matches[worldcup_matches['Away Team Goals']==worldcup_matches['Away Team Goals'].max()]

Fixing team initials, as depreciated countries initials are different than the predecessor countries names, hence making them consistent

Checking Win Conditions values

In [ ]:
worldcup_matches['Win Conditions'].unique()

Found some matches are missing name of the winner, and some are missing the any win conditions, mostly in the group stages.

Fixing Win Conditions column, to contain winning team with the condition of winning if available, and draw if it nobody won

Create Masks

In [ ]:
# Basic score comparison masks
mask_home_win = worldcup_matches['Home Team Goals'] > worldcup_matches['Away Team Goals']
mask_away_win = worldcup_matches['Home Team Goals'] < worldcup_matches['Away Team Goals']
mask_equal_goals = worldcup_matches['Home Team Goals'] == worldcup_matches['Away Team Goals']

# Penalties mask
mask_penalties = mask_equal_goals & worldcup_matches['Win Conditions'].str.contains('penalties', case=False, na=False)
mask_draw = mask_equal_goals & (~worldcup_matches['Win Conditions'].str.contains('penalties', case=False, na=False))

# Teams pattern mask (sort by length descending to match longer names first)
teams_names_sorted = sorted(teams_names, key=len, reverse=True)
teams_pattern = '|'.join([f'\\b{re.escape(team)}\\b' for team in teams_names_sorted])
mask_has_team = worldcup_matches['Win Conditions'].str.contains(teams_pattern, case=False, na=False, regex=True)

# Empty Win Conditions mask
mask_empty_win_cond = worldcup_matches['Win Conditions'].str.strip() == ''

# Non-draw mask
mask_non_draw = ~mask_draw

# Extract penalty scores for penalty matches
worldcup_matches.loc[mask_penalties, 'home_pen_score'] = worldcup_matches.loc[mask_penalties, 'Win Conditions'].str[-6].astype(int)
worldcup_matches.loc[mask_penalties, 'away_pen_score'] = worldcup_matches.loc[mask_penalties, 'Win Conditions'].str[-2].astype(int)

# Create penalty winner masks
mask_pen_home_win = mask_penalties & (worldcup_matches['home_pen_score'] > worldcup_matches['away_pen_score'])
mask_pen_away_win = mask_penalties & (worldcup_matches['home_pen_score'] < worldcup_matches['away_pen_score'])


Assign winners

In [ ]:
# Regular wins
worldcup_matches.loc[mask_home_win, 'winner'] = worldcup_matches.loc[mask_home_win, 'Home Team Name']
worldcup_matches.loc[mask_away_win, 'winner'] = worldcup_matches.loc[mask_away_win, 'Away Team Name']

# Draws
worldcup_matches.loc[mask_draw, 'winner'] = 'Draw'

# Penalty winners
worldcup_matches.loc[mask_pen_home_win, 'winner'] = worldcup_matches.loc[mask_pen_home_win, 'Home Team Name']
worldcup_matches.loc[mask_pen_away_win, 'winner'] = worldcup_matches.loc[mask_pen_away_win, 'Away Team Name']

Update Win Conditions

In [ ]:
# Case 1: Draw
worldcup_matches.loc[mask_draw, 'Win Conditions'] = 'Draw'

# Case 2: Win Conditions contains a team name (non-draw matches)
mask_has_team_non_draw = mask_non_draw & mask_has_team

if mask_has_team_non_draw.any():    
    # Extract the team name that appears at the start
    extracted_team = worldcup_matches.loc[mask_has_team_non_draw, 'Win Conditions'].str.split(
    r'\s+(?:win|on\s+penalties)', 
    n=1, 
    regex=True
    ).str[0].str.strip()
    
    # Compare with winner (case-insensitive)
    matches = (worldcup_matches.loc[mask_has_team_non_draw, 'winner'].str.lower() == 
               extracted_team.str.lower())
    
    # Create mask for mismatches
    mismatch_mask = mask_has_team_non_draw.copy()
    mismatch_mask.loc[mask_has_team_non_draw] = ~matches
    
    if mismatch_mask.any():
        print(f"Found {mismatch_mask.sum()} mismatches")
        print(worldcup_matches.loc[mismatch_mask, ['Home Team Name', 'Away Team Name', 'winner', 'Win Conditions']])
        
        # Remove old team name and prepend correct winner
        worldcup_matches.loc[mismatch_mask, 'Win Conditions'] = (
            worldcup_matches.loc[mismatch_mask, 'winner'] + ' ' + 
            worldcup_matches.loc[mismatch_mask, 'Win Conditions'].str.replace(
                rf'^({teams_pattern})\s*', 
                '', 
                case=False, 
                regex=True
            )
        )
    else:
        print("All marked teams as winning are correct")

# Case 3: Win Conditions doesn't contain team name (non-draw, non-empty)
mask_no_team = mask_non_draw & (~mask_has_team) & (~mask_empty_win_cond)
worldcup_matches.loc[mask_no_team, 'Win Conditions'] = (
    worldcup_matches.loc[mask_no_team, 'winner'] + ' ' + 
    worldcup_matches.loc[mask_no_team, 'Win Conditions']
)

# Case 4: Empty Win Conditions (non-draw)
mask_empty_non_draw = mask_non_draw & mask_empty_win_cond
worldcup_matches.loc[mask_empty_non_draw, 'Win Conditions'] = (
    worldcup_matches.loc[mask_empty_non_draw, 'winner'] + ' win'
)

Cleanup temporary columns

In [ ]:
worldcup_matches.drop(['home_pen_score', 'away_pen_score', 'winner'], axis=1, inplace=True, errors='ignore')

print("Win Conditions updated successfully!")

In [ ]:
worldcup_matches['Win Conditions'].unique()

Exploratory visualizations

In [ ]:
worldcup_matches['Attendance'].plot(kind='hist')

In [ ]:
worldcup_matches['Home Team Goals'].plot(kind='hist')

In [ ]:
worldcup_matches['Away Team Goals'].plot(kind='hist')

In [ ]:
print("DATA CLEANING SUMMARY - WorldCupMatches")

# Record counts
print(f"\n  Dataset Overview:")
print(f"  Total matches: {len(worldcup_matches):,}")
print(f"  Date range: {worldcup_matches['Year'].min()} - {worldcup_matches['Year'].max()}")
print(f"  Unique teams: {len(teams_names)}")
print(f"  Unique stadiums: {worldcup_matches['Stadium'].nunique()}")
print(f"  Unique cities: {worldcup_matches['City'].nunique()}")

# Cleaning actions
print(f"\n  Cleaning Actions Performed:")
print(f"  Null rows removed: 3720")
print(f"  Duplicate matches removed: 16")
print(f"  Missing attendance filled: 1")
print(f"  Country names standardized: 13")
print(f"  Stage names standardized: 10")

# Data quality checks
print(f"\n  Data Quality Verification:")
print(f"  Null values remaining: {worldcup_matches.isnull().sum().sum()}")
print(f"  Duplicate MatchIDs: {worldcup_matches.duplicated(subset=['MatchID']).sum()}")
print(f"  Year/Datetime mismatches: {(worldcup_matches.Datetime.dt.year != worldcup_matches.Year).sum()}")

# Statistics
print(f"\n  Match Statistics:")
print(f"  Total goals scored: {worldcup_matches['Home Team Goals'].sum() + worldcup_matches['Away Team Goals'].sum():,}")
print(f"  Average goals per match: {(worldcup_matches['Home Team Goals'] + worldcup_matches['Away Team Goals']).mean():.2f}")
print(f"  Highest scoring match: {worldcup_matches['Home Team Goals'].max()} - {worldcup_matches['Away Team Goals'].max()}")
print(f"  Total attendance: {worldcup_matches['Attendance'].sum():,}")
print(f"  Average attendance: {worldcup_matches['Attendance'].mean():,.0f}")

# Win conditions breakdown
print(f"\n  Win Conditions Breakdown:")
win_cond_counts = worldcup_matches['Win Conditions'].str.extract(r'(Draw|penalties|extra time)', expand=False).value_counts()
print(f"  Draws: {(worldcup_matches['Win Conditions'] == 'Draw').sum()}")
print(f"  Penalties: {worldcup_matches['Win Conditions'].str.contains('penalties', case=False, na=False).sum()}")
print(f"  Extra time: {worldcup_matches['Win Conditions'].str.contains('extra time', case=False, na=False).sum()}")
print(f"  Regular time: {len(worldcup_matches) - (worldcup_matches['Win Conditions'] == 'Draw').sum() - worldcup_matches['Win Conditions'].str.contains('penalties|extra time', case=False, na=False).sum()}")

# Memory optimization
print(f"\n  Memory Optimization:")
print(f"  Memory usage: {worldcup_matches.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
print(f"  Avg per record: {worldcup_matches.memory_usage(deep=True).sum() / len(worldcup_matches):.0f} bytes")

Export worldCupMatches clean csv under generated directory

In [ ]:
worldcup_matches.to_csv('../data/generated/WorldCupsMatches_Clean.csv', index=False)